# PEEK in PyTorch

## Pretrained VGG Image Classifier Demo

### Run Inference and Save Feature Maps

In [1]:
import torch
import torchvision.models as models

# Load the pretrained VGG-16 model
vgg16 = models.vgg16(pretrained=True)

# If you want to use the model for inference, set it to evaluation mode
vgg16.eval()

# Print the model structure
print(vgg16)

c:\Users\rwhite\AppData\Local\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\rwhite\AppData\Local\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\rwhite/.cache\torch\hub\checkpoints\vgg16-397923af.pth
100%|██████████| 528M/528M [00:36<00:00, 15.1MB/s] 


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

['images/imagenet_valset_images\\ILSVRC2012_val_00000193.JPEG',
 'images/imagenet_valset_images\\ILSVRC2012_val_00000195.JPEG',
 'images/imagenet_valset_images\\ILSVRC2012_val_00000205.JPEG',
 'images/imagenet_valset_images\\ILSVRC2012_val_00000206.JPEG',
 'images/imagenet_valset_images\\ILSVRC2012_val_00000207.JPEG',
 'images/imagenet_valset_images\\ILSVRC2012_val_00000208.JPEG',
 'images/imagenet_valset_images\\ILSVRC2012_val_00000209.JPEG',
 'images/imagenet_valset_images\\ILSVRC2012_val_00000210.JPEG',
 'images/imagenet_valset_images\\ILSVRC2012_val_00000211.JPEG',
 'images/imagenet_valset_images\\ILSVRC2012_val_00000212.JPEG']

In [36]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import pickle
import os

class VGG16FeatureExtractor(torch.nn.Module):
    def __init__(self, pretrained=True):
        super(VGG16FeatureExtractor, self).__init__()
        self.vgg16 = models.vgg16(pretrained=pretrained).features
        # Automatically collect indices of all convolutional layers
        self.conv_layers = [i for i, layer in enumerate(self.vgg16) if isinstance(layer, torch.nn.Conv2d)]

    def forward(self, x):
        features = []
        for layer_index, layer in enumerate(self.vgg16):
            x = layer(x)
            if layer_index in self.conv_layers:
                features.append(x)
        return features

    def load_image(self, image_path):
        # Load an image and transform it to the format required by VGG16
        transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        image = Image.open(image_path)
        image = transform(image).unsqueeze(0)  # Add batch dimension
        return image

    def save_features(self, features, save_folder, base_filename):
        # Save features to a single pickle file
        filename = f'feature_maps/{save_folder}/{base_filename}.pkl'
        with open(filename, "wb") as f:
            pickle.dump([feature.cpu().numpy() for feature in features], f)
        print(f"Saved all features to {filename}")


In [38]:
import glob

image_folder = 'images/imagenet_valset_images'

image_filepaths = glob.glob(f'{image_folder}/*')

feature_extractor = VGG16FeatureExtractor()
feature_extractor.eval()  # Set to evaluation mode

# List of image paths
_, save_folder = image_folder.split('/')

for image_path in image_filepaths:
    input_tensor = feature_extractor.load_image(image_path)
    with torch.no_grad():
        features = feature_extractor(input_tensor)

    # Create a base filename for saving features without the original extension
    base_image_filename = os.path.split(image_path)[-1].split('.')[0]
    feature_extractor.save_features(features, save_folder, base_image_filename)


Saved all features to feature_maps/imagenet_valset_images/ILSVRC2012_val_00000193.pkl
Saved all features to feature_maps/imagenet_valset_images/ILSVRC2012_val_00000195.pkl
Saved all features to feature_maps/imagenet_valset_images/ILSVRC2012_val_00000205.pkl
Saved all features to feature_maps/imagenet_valset_images/ILSVRC2012_val_00000206.pkl
Saved all features to feature_maps/imagenet_valset_images/ILSVRC2012_val_00000207.pkl
Saved all features to feature_maps/imagenet_valset_images/ILSVRC2012_val_00000208.pkl
Saved all features to feature_maps/imagenet_valset_images/ILSVRC2012_val_00000209.pkl
Saved all features to feature_maps/imagenet_valset_images/ILSVRC2012_val_00000210.pkl
Saved all features to feature_maps/imagenet_valset_images/ILSVRC2012_val_00000211.pkl
Saved all features to feature_maps/imagenet_valset_images/ILSVRC2012_val_00000212.pkl
